* [x] Load Data
* [x] Create CNN 
* [x] Train CNN
* [ ] Visualize CNN
* [ ] Train Forecast using CNN
* [ ] Visualize Forecast CNN

In [1]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import DataLoader

from fastai.basic_data import DataBunch
from fastai.basic_train import Learner
from fastai.callbacks import OneCycleScheduler
from fastai.basic_data import DatasetType

from util import convert_ts_data_to_cnn_ts_data,FastAICompatibleDataSet, dev_to_np
from autoencoder import Autoencoder

import datasets, converter


In [2]:
batch_size = 100
max_epoch = 1000

lr = 0.01

shuffle = False
drop_last = True

splits = [0.7,0.2,0.1]


num_jobs=4

device = "cpu"

In [3]:
data = datasets.HessianLoadDataset("../csv",use_cache=False,nr_parallel_file_reader=num_jobs,data_scaler=None)

# put data in variable for vizualization    
#orinial_data = data.original_data    #all data
training_data=data.train_data['targets'][0]
test_data=data.test_data['targets'][0]
validation_data=data.validation_data['targets'][0]


#read in residual data


residual_data =  pd.read_csv(f"residual_load.csv", sep=";", decimal=".",thousands=",", 
                             na_values=["-"], dtype={"Datum":str, "Uhrzeit":str}) 
residual_data.index = pd.to_datetime([f"{residual_data['Datum'][d]} {residual_data['Uhrzeit'][d]}" 
                                         for d in residual_data.index], format="%d.%m.%Y %H:%M:%S")
residual_data = residual_data.fillna(method="bfill")

residual_data = residual_data.drop(['Datum', 'Uhrzeit'],axis=1)

indices_train = pd.to_datetime(pd.date_range(start=data.train_data['targets'][0].index[0], end=data.train_data['targets'][0].index[-1], freq="H"))


train_residual = residual_data.loc[residual_data.index.intersection(indices_train)]['Residuallast']
train_residual=train_residual.resample('1H').asfreq()
train_residual=train_residual.interpolate()

indices_validation = pd.to_datetime(pd.date_range(start=data.validation_data['targets'][0].index[0], end=data.validation_data['targets'][0].index[-1], freq="H"))
validation_residual = residual_data.loc[residual_data.index.intersection(indices_validation)]['Residuallast']

indices_test = pd.to_datetime(pd.date_range(start=data.test_data['targets'][0].index[0], end=data.test_data['targets'][0].index[-1], freq="H"))
test_residual= residual_data.loc[residual_data.index.intersection(indices_test)]['Residuallast']


# create TimeFeatures    'Month of Year', 'Day of Month' , 
                        #'Day of Week' and 'Hour of Day'

TFC=converter.TimeFeatureCreator(year=False,
                                        month_of_year = True,
                                        week_of_year = False,
                                        day_of_year = False,
                                        day_of_month = True,
                                        day_of_week = True,
                                        hour_of_day = True,
                                        minute_of_hour = False,
                                        do_sin_cos_encoding =True)


#write time features into DataFrames, name columns and set index
Time_train=pd.DataFrame(TFC.transform(data.train_data["inputs"][0].index)) #Time Features for train data
Time_train.columns=TFC.extracted_features_name # naming columns
Time_train.index = pd.DatetimeIndex(data.train_data["inputs"][0].index.values, freq='1H') # set index

Time_validation=pd.DataFrame(TFC.transform(data.validation_data["inputs"][0].index)) #time features for validationd ata
Time_validation.columns=TFC.extracted_features_name
Time_validation.index = pd.DatetimeIndex(data.validation_data["inputs"][0].index.values, freq='1H')

Time_test=pd.DataFrame(TFC.transform(data.test_data["inputs"][0].index)) #time features for test data
Time_test.columns=TFC.extracted_features_name
Time_test.index = pd.DatetimeIndex(data.test_data["inputs"][0].index.values, freq='1H')


# Complement Input-Data with Time-Stamp-Data for training, validation and test
# note, that the Input-Data are the same for whole Dataset

input_train=pd.concat([data.train_data["inputs"][0],Time_train], axis=1)
input_validation=pd.concat([data.validation_data["inputs"][0],Time_validation], axis=1)
input_test=pd.concat([data.test_data["inputs"][0],Time_test], axis=1)

# define Reduced-Inputs for training, validation and test
# reduced Input-Data = Temperature, Hour of Day, Day of Week


reduced_features=['temp2m','day_of_week_sin', 'day_of_week_cos', 
                'hour_of_day_sin', 'hour_of_day_cos' ]

input_train_reduced=input_train[reduced_features].copy()
input_validation_reduced=input_validation[reduced_features].copy()
input_test_reduced=input_test[reduced_features].copy()

In [4]:
# data = pd.read_csv('./data/wf1.csv')

# data.index = pd.to_datetime([x.replace('0000','2017').replace('0001','2018') for x in data['Time']])

# data = data.drop(['Time','ForecastingTime'],axis=1)

In [5]:
# def cnn_mse_loss(y_true, y_pred):
#     return np.mean(np.sqrt(np.mean(np.mean((y_pred-y_true)**2,axis=2),axis=0)))

In [6]:
# day_of_x_features = pd.DataFrame(np.array([data.index.month.values, data.index.dayofyear.values, data.index.dayofweek.values, data.index.weekofyear.values]).transpose(), columns=['MonthOfYear','DayOfYear','DayOfWeek','WeekOfYear'], index=data.index)

In [7]:
# all_data = pd.concat([data,day_of_x_features],axis=1)

In [8]:
# all_data = all_data/all_data.max()

In [9]:
# target = all_data['PowerGeneration']
# all_data = all_data.drop(['PowerGeneration'],axis=1)

In [10]:
# split_sizes = np.cumsum([int(x*all_data.shape[0]) for x in splits])

# train_data = convert_ts_data_to_cnn_ts_data(all_data.iloc[0:split_sizes[0]])
# val_data = convert_ts_data_to_cnn_ts_data(all_data.iloc[(split_sizes[1]+1):split_sizes[2]])
# test_data = convert_ts_data_to_cnn_ts_data(all_data.iloc[(split_sizes[0]+1):split_sizes[1]])


In [11]:
# val_data.shape

In [4]:
train_max = input_train.max()

In [16]:
train_max

temp2m                 301.200012
rhum2m                 100.000000
srfprs               99242.960938
cdclow                 100.000000
cdcmid                 100.000000
cdchig                 100.000000
cdctbl                 100.000000
cdctcl                 100.000000
totpre                  10.580000
presno                   1.000000
preice                   0.000000
swfldw                 732.000000
pblhgt                1967.040039
sundur               21600.000000
albedo                  58.700001
liftix                  22.700001
tozone                 470.299988
capest                2671.000000
cfrain                   1.000000
ww30mb                  24.325743
wd30mb                 359.843231
wwsig1                  18.566326
wdsig1                 359.600739
ww010m                  13.814692
wd010m                 359.593658
ww100m                  23.642803
wd100m                 359.795929
month_of_year           12.000000
month_of_year_sin        1.000000
month_of_year_

In [5]:
input_train_scaled = input_train/train_max
input_validation_scaled = input_validation/train_max
input_test_scaled = input_train/train_max

In [22]:
for x in [input_test_scaled,input_validation_scaled,input_train_scaled]:
    x.drop(['preice'],inplace=True, axis=1)

In [25]:
def check_nans(df):
    result = False
    if (type(df) == pd.DataFrame or type(df) == pd.Series):
        df = df.replace([np.inf, -np.inf], np.nan)
        result = df.isnull().any().any()
    elif type(df) == np.ndarray or type(df) == np.array:
        result = np.any(np.isnan(df)) or np.any(np.isinf(df))
    else: 
        result = True
    if (result):
        print("data is null/nan ")
        raise Exception("data is null/nan")
    else:
        return result


In [26]:
for i,input_data in enumerate([input_test, input_train, input_validation, input_train_reduced, input_validation_reduced, input_test_reduced,input_train_scaled,input_validation_scaled,input_test_scaled]):
    print(i)
    if check_nans(input_data):
        print(f"{i}th input data contains nan")
        raise Exception("can't continue with nan in input set")

0
1
2
3
4
5
6
7
8


In [27]:
train_dl = DataLoader(FastAICompatibleDataSet(input_train_scaled,input_train_scaled),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)
val_dl = DataLoader(FastAICompatibleDataSet(input_validation_scaled,input_validation_scaled),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)
test_dl = DataLoader(FastAICompatibleDataSet(input_test_scaled,input_test_scaled),batch_size, shuffle=shuffle,drop_last=drop_last, pin_memory=True if device == "cpu" else False)


In [28]:
# FastAICompatibleDataSet(train_data,train_data)[0:100][1].shape

In [29]:
train_dl.dataset.to_np()[0].shape

(6128, 38)

In [30]:
db = DataBunch(train_dl, val_dl, test_dl=test_dl)

In [31]:
loss = torch.nn.MSELoss()

In [39]:
model = Autoencoder(input_train_scaled.shape[1],[30,20, 10, 5]).to(device)

In [40]:
model

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=38, out_features=30, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=20, out_features=10, bias=True)
    (5): LeakyReLU(negative_slope=0.1)
    (6): Linear(in_features=10, out_features=5, bias=True)
    (7): LeakyReLU(negative_slope=0.1)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=20, out_features=30, bias=True)
    (5): LeakyReLU(negative_slope=0.1)
    (6): Linear(in_features=30, out_features=38, bias=True)
  )
)

In [41]:
def init_model(m,init_function=torch.nn.init.xavier_normal_):
    if type(m) == torch.nn.Linear:
        init_function(m.weight)
        m.bias.data.fill_(0.1)
    if isinstance(m, torch.nn.Conv1d) or isinstance(m, torch.nn.ConvTranspose1d):
        init_function(m.weight.data)
        m.bias.data.fill_(0.1)

In [42]:
model.apply(lambda m: init_model(m))

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=38, out_features=30, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=20, out_features=10, bias=True)
    (5): LeakyReLU(negative_slope=0.1)
    (6): Linear(in_features=10, out_features=5, bias=True)
    (7): LeakyReLU(negative_slope=0.1)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Linear(in_features=10, out_features=20, bias=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=20, out_features=30, bias=True)
    (5): LeakyReLU(negative_slope=0.1)
    (6): Linear(in_features=30, out_features=38, bias=True)
  )
)

In [43]:
learner = Learner(db,model,loss_func=loss)
cb = OneCycleScheduler(learner, lr_max=0.1)


In [44]:
# db.train_ds.

In [ ]:
learner.fit(max_epoch,lr = lr)

epoch,train_loss,valid_loss,time
0,0.133625,0.138405,00:00
1,0.122760,0.126448,00:00
2,0.121991,0.130469,00:00
3,0.123041,0.142195,00:00
4,0.122298,0.134234,00:00
5,0.117517,0.128920,00:00
6,0.111210,0.121044,00:00
7,0.109429,0.117259,00:00
8,0.106276,0.120289,00:00
9,0.108082,0.120357,00:00


In [24]:
y_pred,y_true = learner.get_preds(DatasetType.Valid)
y_pred = dev_to_np(y_pred)
y_true = dev_to_np(y_true)

In [25]:
print(cnn_mse_loss(y_true,y_pred))

0.26652363
